<a href="https://colab.research.google.com/github/simecek/dspracticum2024/blob/main/lesson02/FashionMNIST_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
config = {
    "hidden_layer_width": 128,  # Example value, adjust as needed
    "learning_rate": 0.01,  # Example value, adjust as needed
    "epochs": 20,  # Example value, adjust as needed
    "batch_size": 16,  # Example value, adjust as needed
    "dropout": 0.25,  # Example value, adjust as needed
}

**2. Data Preparation**

In [4]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=config['batch_size'], shuffle=False, )

**3. Neural Network Model**

In [5]:
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, config["hidden_layer_width"])
        self.dropout = nn.Dropout(config['dropout'])
        self.fc12 = nn.Linear(config['hidden_layer_width'], 64)
        self.fc2 = nn.Linear(64, 10)
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc12(x))
        logits = self.fc2(x)
        return logits
    
model = DenseNet().to(device)
model

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc12): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])

**5. Training loop**

In [7]:
for epoch in range(config['epochs']): 
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{config["epochs"]}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/20], Loss: 0.9223
Epoch [2/20], Loss: 0.5332
Epoch [3/20], Loss: 0.4616
Epoch [4/20], Loss: 0.4263
Epoch [5/20], Loss: 0.4047
Epoch [6/20], Loss: 0.3882
Epoch [7/20], Loss: 0.3739
Epoch [8/20], Loss: 0.3616
Epoch [9/20], Loss: 0.3533
Epoch [10/20], Loss: 0.3425
Epoch [11/20], Loss: 0.3355
Epoch [12/20], Loss: 0.3289
Epoch [13/20], Loss: 0.3229
Epoch [14/20], Loss: 0.3166
Epoch [15/20], Loss: 0.3115
Epoch [16/20], Loss: 0.3084
Epoch [17/20], Loss: 0.3012
Epoch [18/20], Loss: 0.2979
Epoch [19/20], Loss: 0.2938
Epoch [20/20], Loss: 0.2897


**6. Evaluation on the test set**

In [8]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 87.10%
